<a href="https://colab.research.google.com/github/dgg32/crop_area/blob/main/Google_Earth_Engine_USA_Crops_Area.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Run me first

First of all, run the following cell to initialize the API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.

In [1]:
from pathlib import Path
import pandas as pd
import ee

In [ ]:

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=k5AyvEpNs0jxyd6mZZfj-FeMvt0UZxQVt00OwzfHLDY&tc=LdQ0BQAy9V97vyB7bwlEGMaLuFygoW4wt7mpTAQijFg&cc=Wf5MOitVLiOAgp6kd_2ev2krN0koUEW_GKoy4H-58eE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJq_bewhqMnYtyb5T6aWVaJKHcbY9Zz92dJi---IrkztlpIkAxk49u4

Successfully saved authorization token.


In [2]:
g_drive_folder = "crop_1e10"

In [ ]:


areaPerPixel = ee.Image.pixelArea()


provinces = ee.FeatureCollection("FAO/GAUL/2015/level2");



def calculate_area (state, crop_index, crop, startDate, endDate):
    def generate_collection(feature):
      geometry = feature.geometry();
      
      dataset = ee.ImageCollection('USDA/NASS/CDL').filter(ee.Filter.date(startDate, endDate)).select("cropland").first()
                  
      o_dataset = dataset.clip(geometry);


      image = dataset.eq(crop_index).selfMask().multiply(areaPerPixel).divide(1e6).set('system:time_start', startDate);
      
      
      reduce = image.select('cropland').reduceRegion(
                                                reducer= ee.Reducer.sum(),
                                                geometry= geometry,
                                                scale= 30,
                                                maxPixels = 1e10
                                                #bestEffort= True
                                            );
            
      return feature.set(crop, reduce.get("cropland"))

    filterLevel1 = ee.Filter.eq('ADM1_NAME', state);
    o_state = provinces.filter(filterLevel1);

    corn_areas = o_state.map(generate_collection)

    results = corn_areas.reduceColumns(ee.Reducer.sum(), [crop])


    return results.values(["sum"]).get(0)

tasks = []

name_list = ee.ImageCollection('USDA/NASS/CDL').select("cropland").first().get("cropland_class_names").getInfo()


crops = ["Corn", "Cotton", "Soybeans"]

state_list = ee.List(["Washington", "Wisconsin", "Wyoming", "Maine", "Michigan", "Minnesota", "Montana", "New Hampshire",
                      "New York", "North Dakota", "South Dakota", "Vermont", "West Virginia", "Alabama", "Alaska", "Arizona",
                      "Arkansas", "California", "Colorado", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas",
                      "Kentucky", "Louisiana", "Maryland", "Mississippi", "Missouri", "Nebraska", "Nevada", "New Mexico", "Ohio",
                      "Oklahoma", "Oregon", "Pennsylvania", "Tennessee", "Texas", "Utah", "Virginia", "Connecticut", "Delaware",
                      "District of Columbia", "Florida", "Massachusetts", "New Jersey", "North Carolina", "Rhode Island", "South Carolina"])

for crop in crops:
    for year in range(2001, 2022):
        startDate = f'{year}-01-01'
        endDate = ee.Date(startDate).advance(1, "year")

        crop_index = name_list.index(crop) + 1;

        results = state_list.map(lambda state: calculate_area(state, crop_index, crop, startDate, endDate)).reduce(ee.Reducer.sum())


        task = ee.batch.Export.table.toDrive(collection=ee.FeatureCollection([ee.Feature(None, {"time": startDate, "crop": crop, "area": results})]),
                                        description=f'{crop}_{startDate}_python_crop',
                                        folder=g_drive_folder,
                                        fileFormat='CSV')
        task.start()

        tasks.append(task)


In [ ]:
for task in tasks:
    print (task.status())
    #task.cancel()

{'state': 'COMPLETED', 'description': 'Corn_2001-01-01_python_crop', 'creation_timestamp_ms': 1664330989068, 'update_timestamp_ms': 1664331082143, 'start_timestamp_ms': 1664330995002, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/#folders/1AXwiaN7PuGTt8m7IjU3Dwib8sw7x_ZR2'], 'attempt': 1, 'batch_eecu_usage_seconds': 35.62931442260742, 'id': '5H5IY3C2CM2KW4ZLYMYVIGOX', 'name': 'projects/earthengine-legacy/operations/5H5IY3C2CM2KW4ZLYMYVIGOX'}
{'state': 'COMPLETED', 'description': 'Corn_2002-01-01_python_crop', 'creation_timestamp_ms': 1664330989367, 'update_timestamp_ms': 1664331094460, 'start_timestamp_ms': 1664330995983, 'task_type': 'EXPORT_FEATURES', 'destination_uris': ['https://drive.google.com/#folders/1AXwiaN7PuGTt8m7IjU3Dwib8sw7x_ZR2'], 'attempt': 1, 'batch_eecu_usage_seconds': 47.38207244873047, 'id': 'SIPZONCOPDCIETSOSQSSCQYR', 'name': 'projects/earthengine-legacy/operations/SIPZONCOPDCIETSOSQSSCQYR'}
{'state': 'COMPLETED', 'description': 'Cor

In [3]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)



Mounted at /content/gdrive/


In [4]:

pathlist = Path(f"gdrive/MyDrive/{g_drive_folder}").glob('*.csv')

rows_list = []

for file in pathlist:
    headers = []
    for line in open(file, 'r'):
        
        if len(headers) == 0:
            headers = line.strip().split(",")
        else:
            temp_dict = {}
            fields = line.strip().split(",")
            for header, cell in zip(headers, fields):
                if header != "system:index" and header != ".geo":
                    temp_dict[header] = cell
            rows_list.append(temp_dict)

df = pd.DataFrame(rows_list)

df.head()

,area,crop,time
0,123435.9264319311,Corn,2001-01-01
1,172110.26377786134,Corn,2002-01-01
2,158982.91259667915,Corn,2003-01-01
3,170490.11507316557,Corn,2004-01-01
4,161194.0117625632,Corn,2005-01-01


In [11]:
df["time"] = pd.to_datetime(df["time"], format='%Y-%m-%d')
df.sort_values(['crop', 'time'], inplace=True)
df.to_csv("crop_area.tsv", sep="\t", index=False)